# Machine Learning Tutorial 
## ESMRMB Lecture, Berlin September 2018
### Teodora Chitiboi, Sandy Engelhardt, Hans Meine 

This tutorial is based on part 1, but uses a CNN Autoencoder.  Let's start with things already known from part 1:

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ibsr_data = np.load('IBSR_v2_resampled_cropped_8bit_64x64.npz')
images = ibsr_data['input']
print('Dimensions and extents after loading:', images.shape)

dataset = np.squeeze(images, axis=1);
print('Dimensions after dimensionality reduction:', dataset.shape) # should be (2716, 64, 64)

# Divide dataset into train and test (order is already random)
x_train = dataset[:2300,:,:]
x_test  = dataset[2300:,:,:]

# Normalizing images to range [0...1]
X_Train = x_train.astype('float32') / 255.
X_Test = x_test.astype('float32') / 255.

## Creating the CNN model architecture

In [ ]:
input_img = Input(shape=(64, 64, 1))  # adapt this if using `channels_first` image data format

# Convolutional layer Kernel 3x3
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
# Max Pooling Kernel 2x2
x = MaxPooling2D((2, 2), padding='same')(x)
# Convolutional layer Kernel 3x3
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# Max Pooling  Kernel 2x2
x = MaxPooling2D((2, 2), padding='same')(x)
# Convolutional layer Kernel 3x3
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# Max Pooling  Kernel 2x2
encoded = MaxPooling2D((2, 2), padding='same')(x)

print('Encoded dimensions should be (8,8,8):', encoded.shape)

# Convolutional layer Kernel 3x3
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
# Upsampling Kernel 2x2
x = UpSampling2D((2, 2))(x)
# Convolutional layer Kernel 3x3
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# Upsampling Kernel 2x2
x = UpSampling2D((2, 2))(x)
# Convolutional layer Kernel 3x3
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
# Upsampling Kernel 2x2
x = UpSampling2D((2, 2))(x)
# Convolutional layer Kernel 3x3
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Set optimizer and loss
autoencoder = Model(input_img, decoded)

In [ ]:
autoencoder.summary()

In [ ]:
# Set the optimizer (Adam is a popular choice), and the loss function
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
# further options: loss='mean_squared_error', 'mean_absolute_error'
# optimizer='sgd'

## Train the autoencoder

In [ ]:
# Potentially change num_epochs or batch_size
num_epochs = 50
autoencoder.fit(x_train, x_train,
                epochs=num_epochs,
                batch_size=16,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Test the autoencoder using the model to predict unseen data
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
# Following code is for displaying of results

n = 6 # number of images to display
plt.figure(figsize=(12, 4))
for i in range(n):
    # display image
    ax = plt.subplot(2, n, i + 1)
    ax.imshow(x_test[i].reshape(64, 64), cmap = 'gray')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect(1.0)

    # display reconstructed
    ax = plt.subplot(2, n, i + 1 + n)
    ax.imshow(decoded_imgs[i].reshape(64, 64), cmap = 'gray')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect(1.0)
plt.show()